# Modelling

## Outline of the Process

1. Build (two) embedding models to represent both items (games) and users (steam users) as an embedding vector.
2. Find the similarity between game and user vectors
    - Use distance metrics (cosine similarity, euclidean distance, etc.) to determine their similarity score
3. Rank games by their similarity score with the user
4. Remove games already owned in their library
5. Present top 6 games from the remaining list of ranked games.

## Benefits (Business worth)

This method of finding recommendations overcomes the so-called "Cold Start Problem": it is hard to predict a user's interests when they are new users. By learning the embeddings of existing users and items, the system can make initial recommenations for new users or provide recommendations for new items based on their similarity to existing items.

## Considerations for Modelling

There are many types of recommender systems 


### Types of Models


#### 1. Simple KNN

#### 2. Item-Item Filtering

#### 3. Item-User Filtering

#### 4. Others?

#### 5. NN

### Accuracy Concerns

The model's accuracy can be a valuable metric to determine if the recommendations being given to a user are appropriate.  An important consideration, however, is our tolerance for inaccurate recommendations making it into our top recommended list.  For instance, how much should we care if the system provides 5 good recommendations but 1 very bad recommendation compared to 6 mediocre recommendations?  In other words, how important is the system's ability to prevent false positives (i.e., its precision)?

For my recommender system, I will give high preference to precision over accuracy since I want to prevent false positives whenever possible.

### 0. Preparation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.neighbors import NearestNeighbors

from warnings import filterwarnings
filterwarnings('ignore')

ModuleNotFoundError: No module named 'sklearn'

### 1. KNN